# Assignment 4

## Task 1: 

### Inverted list: 

In [18]:
from collections import defaultdict
import re

# Sample text
text = "Ocean environment is a deep and complex ecosystem filled with diverse creatures. Yet, much about the ocean lacks exploration."

# Define stop words. Based on the text, we chose some common stop words.
stop_words = {"is", "a", "and", "with", "the", "about", "yet"}

# Step 1: Tokenize text and filter out stop words
words = [word.lower() for word in re.findall(r'\w+', text) if word.lower() not in stop_words]

print(" (a): Text after removing stop-words:")
print(words)

# Step 2: Set block size for block addressing
# We chose a block size of 2 for this example. The block size is arbitrary and depends on balancing precision and storage efficiency.
# Since this is a fairly small text, we chose a smaller block size of 2, which provides higher precision in locating terms and doesn't affect storage efficiency.
block_size = 2

# Step 3a: Build the original inverted index with individual positions
inverted_index_positions = defaultdict(list)
for pos, word in enumerate(words):
    inverted_index_positions[word].append(pos)

# Step 3b: Build the inverted index with block addressing
inverted_index_block = defaultdict(set)
for pos, word in enumerate(words):
    # Calculate the block number for each position
    block_num = pos // block_size
    inverted_index_block[word].add(block_num)

# Convert sets to lists for final output format
inverted_index_block = {word: sorted(list(blocks)) for word, blocks in inverted_index_block.items()}

# Display both inverted indexes
print("\nInverted index with individual positions")
print(dict(inverted_index_positions))  # Convert defaultdict to dict for clean display

print("\n (b): Inverted index with block addressing")
print(inverted_index_block)




 (a): Text after removing stop-words:
['ocean', 'environment', 'deep', 'complex', 'ecosystem', 'filled', 'diverse', 'creatures', 'much', 'ocean', 'lacks', 'exploration']

Inverted index with individual positions
{'ocean': [0, 9], 'environment': [1], 'deep': [2], 'complex': [3], 'ecosystem': [4], 'filled': [5], 'diverse': [6], 'creatures': [7], 'much': [8], 'lacks': [10], 'exploration': [11]}

 (b): Inverted index with block addressing
{'ocean': [0, 4], 'environment': [0], 'deep': [1], 'complex': [1], 'ecosystem': [2], 'filled': [2], 'diverse': [3], 'creatures': [3], 'much': [4], 'lacks': [5], 'exploration': [5]}


 (c): Partial vocabulary suffix tree

In [19]:
class SuffixTreeNode:
    def __init__(self):
        self.children = {}
        self.is_end_of_suffix = False

class SuffixTree:
    def __init__(self):
        self.root = SuffixTreeNode()

    def insert_suffix(self, suffix):
        node = self.root
        for char in suffix:
            if char not in node.children:
                node.children[char] = SuffixTreeNode()
            node = node.children[char]
        node.is_end_of_suffix = True

    def build_suffix_tree(self, word):
        for i in range(len(word)):
            self.insert_suffix(word[i:])

    def display_tree(self, node=None, prefix=''):
        if node is None:
            node = self.root
        for char, child in node.children.items():
            print(prefix + char)
            self.display_tree(child, prefix + char)

# Example usage
suffix_tree = SuffixTree()
suffix_tree.build_suffix_tree("ocean")
suffix_tree.build_suffix_tree("environment")
suffix_tree.build_suffix_tree("explore")

print("Partial Suffix Tree:")
suffix_tree.display_tree()


Partial Suffix Tree:
o
oc
oce
ocea
ocean
on
onm
onme
onmen
onment
or
ore
c
ce
cea
cean
e
ea
ean
en
env
envi
envir
enviro
environ
environm
environme
environmen
environment
ent
ex
exp
expl
explo
explor
explore
a
an
n
nv
nvi
nvir
nviro
nviron
nvironm
nvironme
nvironmen
nvironment
nm
nme
nmen
nment
nt
v
vi
vir
viro
viron
vironm
vironme
vironmen
vironment
i
ir
iro
iron
ironm
ironme
ironmen
ironment
r
ro
ron
ronm
ronme
ronmen
ronment
re
m
me
men
ment
t
x
xp
xpl
xplo
xplor
xplore
p
pl
plo
plor
plore
l
lo
lor
lore


 (c): Partial vocabulary tree

In [20]:
class TrieNode:
    def __init__(self):
        self.children = {}
        self.is_end_of_word = False

class VocabularyTree:
    def __init__(self):
        self.root = TrieNode()

    def insert_word(self, word):
        node = self.root
        for char in word:
            if char not in node.children:
                node.children[char] = TrieNode()
            node = node.children[char]
        node.is_end_of_word = True

    def display_tree(self, node=None, prefix=''):
        if node is None:
            node = self.root
        for char, child in node.children.items():
            print(prefix + char)
            self.display_tree(child, prefix + char)

# Example usage
vocab_tree = VocabularyTree()
vocab_tree.insert_word("ocean")
vocab_tree.insert_word("environment")
vocab_tree.insert_word("explore")

print("\nPartial Vocabulary Tree:")
vocab_tree.display_tree()



Partial Vocabulary Tree:
o
oc
oce
ocea
ocean
e
en
env
envi
envir
enviro
environ
environm
environme
environmen
environment
ex
exp
expl
explo
explor
explore


### Posting list

In [21]:
from collections import defaultdict
import re

# Define the document collection
documents = {
    "D1": "The universe is vast and mysterious, filled with countless stars and galaxies.",
    "D2": "Each star may host its own planets, some of which could harbor life.",
    "D3": "Scientists explore these possibilities through advanced telescopes and technology.",
    "D4": "Yet, many questions about our universe remain unanswered, driving curiosity and research.",
    "D5": "The quest for knowledge continues, pushing the boundaries of human understanding."
}

# We defined a small set of common stop words to filter out, based on the provided documents.
stop_words = {"the", "is", "and", "its", "of", "for", "with", "a", "may"}

# Create an empty inverted index (defaultdict to handle missing keys)
inverted_index = defaultdict(list)

# Tokenize and index each document
for doc_id, text in documents.items():
    # Tokenize text by splitting on non-alphanumeric characters, convert to lowercase
    words = re.findall(r'\w+', text.lower())
    # Filter out stop words and add words to the index
    for word in words:
        if word not in stop_words:
            # Append the document ID to the term's posting list if it's not already present
            if doc_id not in inverted_index[word]:
                inverted_index[word].append(doc_id)

# Display the inverted index
print("Inverted Index with Posting Lists:")
for term, posting_list in inverted_index.items():
    print(f"{term}: {posting_list}")

Inverted Index with Posting Lists:
universe: ['D1', 'D4']
vast: ['D1']
mysterious: ['D1']
filled: ['D1']
countless: ['D1']
stars: ['D1']
galaxies: ['D1']
each: ['D2']
star: ['D2']
host: ['D2']
own: ['D2']
planets: ['D2']
some: ['D2']
which: ['D2']
could: ['D2']
harbor: ['D2']
life: ['D2']
scientists: ['D3']
explore: ['D3']
these: ['D3']
possibilities: ['D3']
through: ['D3']
advanced: ['D3']
telescopes: ['D3']
technology: ['D3']
yet: ['D4']
many: ['D4']
questions: ['D4']
about: ['D4']
our: ['D4']
remain: ['D4']
unanswered: ['D4']
driving: ['D4']
curiosity: ['D4']
research: ['D4']
quest: ['D5']
knowledge: ['D5']
continues: ['D5']
pushing: ['D5']
boundaries: ['D5']
human: ['D5']
understanding: ['D5']


## Task 2:
task 2: (b) and (c) 

In [ ]:
import os
import re
from collections import defaultdict

# Task b: Implementation of a simple text indexer. 
class SimpleTextIndexer:
    def __init__(self):
        # Load stop words from a file
        with open('common-english-words.txt', 'r') as file:
            # Assuming stop words are comma-separated in the file
            stopwords = set(file.read().strip().split(","))
        
        # Create an inverted index: {word: [doc_ids]}
        self.inverted_index = defaultdict(set)
        self.stop_words = stopwords  # Use the loaded stop words
    
    def index_document(self, doc_id, text):
        # Tokenize text
        words = re.findall(r'\w+', text.lower())  # Lowercase to ensure case-insensitive search
        for word in words:
            # Ignore stop words
            if word not in self.stop_words:
                self.inverted_index[word].add(doc_id)
    
    def search(self, query):
        # Split the query and find documents containing each word
        query_words = query.lower().split()
        results = [self.inverted_index[word] for word in query_words if word in self.inverted_index]
        
        # Return documents containing all query words (intersection)
        if results:
            return set.intersection(*results)
        return set()

# Task c: Indexing documents provided in assignment.
# Function to index documents from a list
def index_documents_from_list(file_list, indexer, directory):
    # Go through each file in the provided list and index it
    for file_name in file_list:
        file_path = os.path.join(directory, file_name)
        if os.path.isfile(file_path):  # Check if the file exists
            try:
                with open(file_path, 'r', encoding='utf-8') as file:
                    text = file.read()
                    indexer.index_document(file_name, text)
                    print(f"Indexed {file_name} successfully.")
            except Exception as e:
                print(f"An error occurred while reading {file_name}: {e}")
        else:
            print(f"File {file_name} not found at path: {file_path}")

# Define the path to your DataAssignment4 directory
data_directory = os.path.join(os.getcwd(), "DataAssignment4")
textFiles = ["Text1.txt", "Text2.txt", "Text3.txt", "Text4.txt", "Text5.txt", "Text6.txt"]

# Main execution for indexing documents
indexer = SimpleTextIndexer()
index_documents_from_list(textFiles, indexer, data_directory)

# Optionally, print the inverted index for verification
print("\nInverted Index:")
for word, doc_ids in indexer.inverted_index.items():
    print(f"{word}: {doc_ids}")


Indexed Text1.txt successfully.
Indexed Text2.txt successfully.
Indexed Text3.txt successfully.
Indexed Text4.txt successfully.
Indexed Text5.txt successfully.
Indexed Text6.txt successfully.

Inverted Index:
wonderful: {'Text1.txt'}
serenity: {'Text1.txt'}
taken: {'Text1.txt'}
possession: {'Text1.txt'}
entire: {'Text1.txt'}
soul: {'Text1.txt'}
sweet: {'Text1.txt'}
mornings: {'Text1.txt'}
spring: {'Text1.txt'}
enjoy: {'Text6.txt', 'Text1.txt'}
whole: {'Text2.txt', 'Text1.txt'}
heart: {'Text1.txt'}
alone: {'Text1.txt'}
feel: {'Text2.txt', 'Text1.txt'}
charm: {'Text1.txt'}
existence: {'Text5.txt', 'Text1.txt'}
spot: {'Text2.txt', 'Text1.txt'}
created: {'Text1.txt'}
bliss: {'Text1.txt'}
souls: {'Text1.txt'}
mine: {'Text5.txt', 'Text1.txt'}
happy: {'Text1.txt'}
friend: {'Text5.txt', 'Text1.txt'}
absorbed: {'Text1.txt'}
exquisite: {'Text1.txt', 'Text3.txt'}
sense: {'Text1.txt'}
mere: {'Text1.txt'}
tranquil: {'Text1.txt'}
neglect: {'Text1.txt'}
talents: {'Text1.txt'}
incapable: {'Text1.txt'}